<a href="https://colab.research.google.com/github/OcSpice/Churn-Simple-Linear-Regression/blob/main/Titanic_Data_Cleaning_(Train_%26_Test).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Titanic Data Cleaning & Preprocessing**

## 1️⃣ **Title & Objective**

##### **Level 1 – Task 1: Data Cleaning & Preprocessing (Titanic Dataset)**

##### 🎯 **Objective**
To clean and preprocess both **train.csv** and **test.csv** by:  
- Identifying and handling **missing values**  
- Removing **duplicate records**  
- Standardizing **categorical formats**
- Ensuring both datasets are aligned for future analysis/modeling

## 2️⃣ **Import Libraries & Load Data**

I imported pandas, numpy, matplotlib, and seaborn. These libraries are essential for handling data, exploring missing values and basic visualization.

In [253]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

I loaded both `train.csv` and `test.csv` files from the Titanic dataset. This ensured consistency in cleaning across both datasets.

This datasets contains information about passengers and whether they survived the Titanic disaster.

In [254]:
import pandas as pd
# Load Titanic train & test datasets
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [255]:
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (891, 12)
Test shape: (418, 11)


In [231]:
train_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,Mr
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,Mrs
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,Miss
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,Mrs
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,Mr


## 3️⃣ **Data Cleaning & Inspecting Missing Values**

#### **Step 1: Data Cleaning**
- Dropped irrelevant ID columns.  
- Handled missing values (e.g., Age, Cabin, Embarked).  
- Standardized categorical values for consistency.  

**Missing Value Overview**
- **Train.csv**: Age (177), Cabin (687), Embarked (2)  
- **Test.csv**: Age (86), Cabin (327), Fare (1)

In [256]:
# Extract Title
if 'Name' in train_df.columns:
    train_df.loc[:, 'Title'] = train_df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)

if 'Name' in test_df.columns:
    test_df.loc[:, 'Title'] = test_df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)

# Group rare titles
if 'Title' in train_df.columns:
    rare_titles_train = train_df['Title'].value_counts()[train_df['Title'].value_counts() < 10].index
    train_df.loc[:, 'Title'] = train_df['Title'].replace(rare_titles_train, 'Rare')

if 'Title' in test_df.columns:
    rare_titles_test = test_df['Title'].value_counts()[test_df['Title'].value_counts() < 10].index
    test_df.loc[:, 'Title'] = test_df['Title'].replace(rare_titles_test, 'Rare')

if 'PassengerId' in train_df.columns and 'Name' in train_df.columns and 'Ticket' in train_df.columns:
    train_df = train_df.drop(columns=['PassengerId', 'Ticket'])
if 'PassengerId' in test_df.columns and 'Name' in test_df.columns and 'Ticket' in test_df.columns:
    test_df = test_df.drop(columns=['PassengerId', 'Ticket'])

In [257]:
print("Missing values in Train:\n", train_df.isnull().sum())
print("\nMissing values in Test:\n", test_df.isnull().sum())

Missing values in Train:
 Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
Title         0
dtype: int64

Missing values in Test:
 Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Fare          1
Cabin       327
Embarked      0
Title         0
dtype: int64


In [258]:
# Summary statistics
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Pclass,418.0,NaN,NaN,NaN,2.26555,0.841838,1.0,1.0,3.0,3.0,3.0
Sex,418,2,male,266,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,332.0,NaN,NaN,NaN,30.27259,14.181209,0.17,21.0,27.0,39.0,76.0
SibSp,418.0,NaN,NaN,NaN,0.447368,0.89676,0.0,0.0,0.0,1.0,8.0
Parch,418.0,NaN,NaN,NaN,0.392344,0.981429,0.0,0.0,0.0,0.0,9.0
Fare,417.0,NaN,NaN,NaN,35.627188,55.907576,0.0,7.8958,14.4542,31.5,512.3292
Cabin,91,76,B57 B59 B63 B66,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Embarked,418,3,S,270,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 4️⃣ **Handle Missing Values**

#### **Step 2: Handling Missing Values**
- Filled **Age** with median in both train & test.  
- Filled **Embarked** with mode in train.  
- Filled **Fare** with median in test.  
- Dropped **Cabin** in both due to excessive missing values.

In [259]:
# Train set - fill missing Age with median
train_df.loc[:, 'Age'] = train_df['Age'].fillna(train_df['Age'].median())

In [260]:
train_df['Sex'] = train_df['Sex'].fillna(train_df['Sex'].mode()[0])

In [261]:
# Check mode
print(train_df['Embarked'].mode())

# Fill missing values safely
train_df.loc[:, 'Embarked'] = train_df['Embarked'].fillna(train_df['Embarked'].mode()[0])

0    S
Name: Embarked, dtype: object


In [262]:
# Test set
test_df.loc[:, 'Age'] = test_df['Age'].fillna(test_df['Age'].median())
test_df.loc[:, 'Fare'] = test_df['Fare'].fillna(test_df['Fare'].median())

In [263]:
# Drop 'Cabin' from both train and test (too many missing values)
if 'Cabin' in train_df.columns:
    train_df = train_df.drop(columns=['Cabin'])

if 'Cabin' in test_df.columns:
    test_df = test_df.drop(columns=['Cabin'])

## 5️⃣ **Remove Duplicates**

#### **Step 3: Duplicate Removal**
Checked for duplicate rows in both datasets and removed them (if any).

In [264]:
print("Duplicates before:", train_df.duplicated().sum(), test_df.duplicated().sum())

Duplicates before: 0 0


In [265]:
train_df = train_df.drop_duplicates()
test_df = test_df.drop_duplicates()

In [266]:
print("Duplicates after:", train_df.duplicated().sum(), test_df.duplicated().sum())

Duplicates after: 0 0


## 6️⃣ **Standardize & Encode Categorical Features**

#### **Step 4: Standardizing Formats**
- Converted `Sex` to lowercase in both datasets.  
- Standardized `Embarked` codes in train dataset.  
- Converted `Survived` to categorical in train dataset.

In [268]:
# Convert 'Sex' to lowercase
if 'Sex' in train_df.columns:
    train_df['Sex'] = train_df['Sex'].astype(str).str.lower()

if 'Sex' in test_df.columns:
    test_df['Sex'] = test_df['Sex'].astype(str).str.lower()

In [269]:
train_df['Sex'] = train_df['Sex'].str.lower()
test_df['Sex'] = test_df['Sex'].str.lower()

In [270]:
# Ensure 'Embarked' uppercase (only in train)
if 'Embarked' in train_df.columns:
    train_df['Embarked'] = train_df['Embarked'].astype(str).str.upper()

In [271]:
# Convert 'Survived' to categorical in train
train_df['Survived'] = train_df['Survived'].astype('category')

**Encode Categorical Features**
- Map binary categories (Sex: male=0, female=1).

- One-hot encode multi-category features (Embarked).

In [272]:
# Encode Sex (male=0, female=1)
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})

## 7️⃣ **Feature Engineering**

## **Step 5:**
Created new features to improve dataset quality:  
- Extracted `Title` from passenger names (e.g., Mr, Mrs, Miss).  
- Binned `Age` into groups (Child, Adult, Elderly).  
- Converted `Fare` into categories (Low, Medium, High).

In [273]:
# Bin Age
train_df['AgeGroup'] = pd.cut(train_df['Age'], bins=[0,12,18,50,80], labels=['Child','Teen','Adult','Elder'])
test_df['AgeGroup'] = pd.cut(test_df['Age'], bins=[0,12,18,50,80], labels=['Child','Teen','Adult','Elder'])

# Bin Fare
train_df['FareGroup'] = pd.qcut(train_df['Fare'], 4, labels=['Low','Medium','High','VeryHigh'])
test_df['FareGroup'] = pd.qcut(test_df['Fare'], 4, labels=['Low','Medium','High','VeryHigh'])

## 8️⃣ **Standardize Data Types**

### **Step 6**
Ensured numerical columns (Age, Fare) were floats/integers.  
Categorical features were converted into category types for efficiency.

In [274]:
# Ensure numerical features are floats
train_df['Age'] = train_df['Age'].astype(float)
train_df['Fare'] = train_df['Fare'].astype(float)
test_df['Age'] = test_df['Age'].astype(float)
test_df['Fare'] = test_df['Fare'].astype(float)

## 9️⃣ **Final Checks**

### **Step 7: ✅ Final Clean Datasets**
Both **train.csv** and **test.csv** Titanic dataset has been cleaned and reprocessed:
- Confirmed consistent formatting across both train and test sets.
- Missing values handled
- Duplicates removed
- Irrelevant columns dropped  
- Categorical variables encoded  
- New features engineered
- Saved the cleaned datasets for further analysis or modeling.

This completed the **Data Cleaning & Processing** phase of the Titanic dataset.

In [276]:
# Verify missing values
print("Missing values in Train:\n", train_df.isnull().sum())
print("\nMissing values in Test:\n", test_df.isnull().sum())

# Check final data
print("\nTrain Head:")
display(train_df.head())
print("\nTest Head:")
display(test_df.head())

# Print
print("\nTrain Info:")
train_df.info()
print("\nTest Info:")
test_df.info()

Missing values in Train:
 Survived     0
Pclass       0
Name         0
Sex          0
Age          0
SibSp        0
Parch        0
Fare         0
Embarked     0
Title        0
AgeGroup     0
FareGroup    0
dtype: int64

Missing values in Test:
 Pclass       0
Name         0
Sex          0
Age          0
SibSp        0
Parch        0
Fare         0
Embarked     0
Title        0
AgeGroup     0
FareGroup    0
dtype: int64

Train Head:


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeGroup,FareGroup
0,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,7.2500,S,Mr,Adult,Low
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,71.2833,C,Mrs,Adult,VeryHigh
2,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,7.9250,S,Miss,Adult,Medium
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,53.1000,S,Mrs,Adult,VeryHigh
4,0,3,"Allen, Mr. William Henry",0,35.0,0,0,8.0500,S,Mr,Adult,Medium



Test Head:


,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeGroup,FareGroup
0,3,"Kelly, Mr. James",0,34.5,0,0,7.8292,Q,Mr,Adult,Low
1,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,7.0000,S,Mrs,Adult,Low
2,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,9.6875,Q,Mr,Elder,Medium
3,3,"Wirz, Mr. Albert",0,27.0,0,0,8.6625,S,Mr,Adult,Medium
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,12.2875,S,Mrs,Adult,Medium



Train Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Survived   891 non-null    category
 1   Pclass     891 non-null    int64   
 2   Name       891 non-null    object  
 3   Sex        891 non-null    int64   
 4   Age        891 non-null    float64 
 5   SibSp      891 non-null    int64   
 6   Parch      891 non-null    int64   
 7   Fare       891 non-null    float64 
 8   Embarked   891 non-null    object  
 9   Title      891 non-null    object  
 10  AgeGroup   891 non-null    category
 11  FareGroup  891 non-null    category
dtypes: category(3), float64(2), int64(4), object(3)
memory usage: 65.9+ KB

Test Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Pclass     418 non-null    i

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
